<a href="https://colab.research.google.com/github/EdBerg21/AI-Professional-Prompts/blob/main/03_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import json
from pathlib import Path
import pickle
from tqdm.auto import tqdm

from haystack.nodes.preprocessor import PreProcessor

In [ ]:
proj_dir = Path.cwd().parent
print(proj_dir)

/home/ec2-user/arabic-wiki


# Config

In [ ]:
files_in = list((proj_dir / 'data/consolidated').glob('*.ndjson'))
folder_out = proj_dir / 'data/processed'
folder_out_str = str(folder_out)

# Analysis

In [ ]:
with open(files_in[0], 'r') as f:
    articles = [json.loads(line) for line in f]

In [ ]:
from pprint import pprint
article = articles[0].copy()
article['content'] = article['content'][:50] + '...'
pprint(article)

{'content': 'الماء مادةٌ شفافةٌ عديمة اللون والرائحة، وهو المكو...',
 'meta': {'id': '7',
          'revid': '2080427',
          'title': 'ماء',
          'url': 'https://ar.wikipedia.org/wiki?curid=7'}}


# Preprocessing

Its important to choose good pre-processing options.

Clean whitespace helps each stage of RAG. It adds noise to the embeddings, and wastes space when we prompt with it.

I chose to split by word as it would be tedious to tokenize here, and that doesnt scale well. The context length for most embedding models ends up being 512 tokens. We saw this within a good z-score is ~225 token.

I like to respect the sentence boundary, thats why I gave a ~50 word buffer.

In [ ]:
pp = PreProcessor(clean_whitespace = True,
             clean_header_footer = False,
             clean_empty_lines = True,
             remove_substrings = None,
             split_by='word',
             split_length = 225,
             split_overlap = 50,
             split_respect_sentence_boundary = True,
             tokenizer_model_folder = None,
             id_hash_keys = None,
             progress_bar = False,
             add_page_number = False,
             max_chars_check = 10_000)

When we break a wikipedia article up, we lose some of the context. The local context is somewhat preserved by the `split_overlap`. Im trying to preserve the global context by adding a prefix that has the article's title.

You could enhance this with the summary as well. This is mostly to help the retrieval step of RAG. Note that the way Im doing it alters some of `haystack`'s features like the hash and the lengths, but those arent too necessary.

A more advanced way for many business applications would be to summarize the document and add that as a prefix for sub-documents.

One last thing to note, is that it would be prudent (in some use-cases) to preserve the original document without the summary to give to the reader (retrieve with the summary but prompt without), but since this is a demo use-case I wont be doing that.

In [ ]:
with open(files_in[0], 'r', encoding='utf-8') as f:
    articles = [json.loads(line) for line in f]

In [ ]:
%%time
documents = pp.process(articles)

We found one or more sentences whose word count is higher than the split length.
Document e3e2bf8b3399979cb16219b175041b4d is 11336 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Document 91ad1d1a24e93abacabd5a5478a96977 is 14251 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Document 1625c431c0fcfaf81c13e0da59071a81 is 13395 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Document 790d3b2

CPU times: user 3min 31s, sys: 95.1 ms, total: 3min 31s
Wall time: 3min 31s


In [ ]:
%%time
import os
import concurrent.futures

def parallel_preprocessing(articles):
    # Utility function to divide the articles into smaller chunks
    def chunkify(lst, n):
        """Yield successive n-sized chunks from lst."""
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # Size of each chunk. Adjust based on your needs.
    CHUNK_SIZE = 10_000
    article_chunks = list(chunkify(articles, CHUNK_SIZE))

    # Number of processes to run in parallel.
    # Use all available CPUs, but you can reduce the number if you wish to leave some CPUs free.
    NUM_PROCESSES = os.cpu_count()

    with concurrent.futures.ProcessPoolExecutor(max_workers=NUM_PROCESSES) as executor:
        documents_list = list(executor.map(pp.process, article_chunks))

    # Flatten the documents_list to get a single list of documents
    documents = [doc for sublist in documents_list for doc in sublist]
    return documents

documents = parallel_preprocessing(articles)


Document 91ad1d1a24e93abacabd5a5478a96977 is 14251 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Document e3e2bf8b3399979cb16219b175041b4d is 11336 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Document 1625c431c0fcfaf81c13e0da59071a81 is 13395 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Document 790d3b2d94a68cbec6d77f3c15d0e679 is 13484 characters long after preprocessing, where the

CPU times: user 6.86 s, sys: 1.31 s, total: 8.16 s
Wall time: 1min 33s


In [ ]:
%%time
for file_in in tqdm(files_in):
    # Load articles
    with open(file_in, 'r', encoding='utf-8') as f:
        articles = [json.loads(line) for line in f]

    # Preprocess articles
    documents = parallel_preprocessing(articles)

    # Prefix each document's content
    for document in tqdm(documents):
        if document.meta['_split_id'] != 0:
            document.content = f'عنوان: {document.meta["title"]}. ' + document.content

    processed_articles = [document.to_dict() for document in documents]
    with open(folder_out/file_in.name, 'w', encoding='utf-8') as f:
        for article in processed_articles:
            json_str = json.dumps(article, ensure_ascii=False)
            f.write(json_str + '\n')


  0%|          | 0/23 [00:00<?, ?it/s]

Document 91ad1d1a24e93abacabd5a5478a96977 is 14251 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Document e3e2bf8b3399979cb16219b175041b4d is 11336 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Document 1625c431c0fcfaf81c13e0da59071a81 is 13395 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Document 790d3b2d94a68cbec6d77f3c15d0e679 is 13484 characters long after preprocessing, where the

  0%|          | 0/243068 [00:00<?, ?it/s]

Document 2693d73124e824fb45633e34189a6226 is 14375 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.


  0%|          | 0/104065 [00:00<?, ?it/s]

...

  0%|          | 0/124472 [00:00<?, ?it/s]

  0%|          | 0/121849 [00:00<?, ?it/s]

  0%|          | 0/147110 [00:00<?, ?it/s]

  0%|          | 0/70322 [00:00<?, ?it/s]

CPU times: user 2min 21s, sys: 20.1 s, total: 2min 41s
Wall time: 13min 36s


## Pre-processing Examples

In [ ]:
documents[0]

<Document: {'content': 'عشاء كارين هو من سلسلة مطاعم استرالية يهدف عمداً عن تجربة تناول طعام غير سارَة ويتم توجيه الموظفين لإهانة العملاء طوال وجباتهم.\nاقتبس اسم المطعم من المصطلح العامي على الإنترنت (كارين) والذي يستخدم لوصف امرأة بيضاء مسنة وقحة بشكل نمطي.\nتاريخ المطعم.\nتم إنشاء السلسلة في أستراليا (سيدني) في عام 2021 من قبل إيدين ليفن وجيمس فاريل. المطعم ذو طابع خاص يعتمد على خدمة تجربة طعام غير سارة حيث يدفع العملاء للموظفين لإهانتهم وكان من المفترض ان يكون المطعم مطعماً منبثقاً لمدة ستة أشهر في وورلد سكوير.\nاثارت فكرة المطعم في البداية ردات فعل متغايرة مما أثار الخوف بشأن ما إذا كانت الإهانات المتبادلة من الممكن ان تعرض الموظفين لسوء المعاملة من قبل العملاء.\nاسم (كارين) هو إشارة إلى الإسم المستخدم في الميمات (النكت التي تنشهر بسرعة في مواقع التواصل) لوصف امرأة بيضاء في منتصف العمر ووقحة بشكل نمطي.\nيطلب من الموظفين ارتداء شخصية وقحة والسخرية من العملاء بشكل هزلي اثناء تناول وجباتهم ومن المتوقع ان يعيد العملاء هذا السلوك من خلال التصرف بوقاحة تجاه الموظفين ومع ذلك يُحظر على ال

In [ ]:
documents[1]

<Document: {'content': 'عنوان: مطعم عشاء كارين. يطلب من الموظفين ارتداء شخصية وقحة والسخرية من العملاء بشكل هزلي اثناء تناول وجباتهم ومن المتوقع ان يعيد العملاء هذا السلوك من خلال التصرف بوقاحة تجاه الموظفين ومع ذلك يُحظر على العملاء والموظفين استخدام الإهانات العنصرية أو التحيز الجنسي أو رهاب المثلية الجنسية.\nتتضمن العديد من هذه التبادلات لغة نابية ويجب ان يكون برفقة الاشخاص اللذين يقلون عن 16 عاماََ بالغين.\nكما يمكن لمالكي بطاقة هوية تظهر ان اسمهم كارين الحصول على مشروب مجاني.\nيرتكز المطعم علو وجبات العشاء الأمريكية في خمسينات القرن الماضي وتتميز القائمة بالهامبرغر وأجنحة الدجاج.\nأصبح محتوى شائع لوسائل التواصل الإجتماعي خصوصاً على منصة (تيك توك) حيث نشر العملاء مقاطع فيديو لتفاعلاتهم مع الموظفين.\nفتحت السلسلة في مواقع في المملكة المتحدة والولايات المتحدة ونيوزيلندا.\nفي شهر أغسطس سنة 2022 اثار المطعم جدلاً بعد ان انتشر مقطع يُظهر فيه أحد موظفي فريق العمل في منطقة بريزبين يتصرف بشكل غير لائق على منصة تيك توك حيث القى تعليقات غير لائقة موجهة إلى زبونة قاصر ووالدها الذي كان يشاركها

In [ ]:
documents[10102]

<Document: {'content': 'نيكولاي فليفا والمعروف أيضًا باسم نيكو فليفا (1840 - 4 أغسطس عام 1920) هو سياسي وصحفي سياسي ومحامي أفلاقي ومن ثم روماني. اشتهر بانخراطه في المجريات السياسية ونزعته الوطنية الصريحة التي كادت تصل إلى حد الديماغوجية. اختبر كافة الصيغ السياسية التي يسمح بها نظام الحزبين في رومانيا. دام حضوره على الساحة العامة عقودًا من الزمن، شغل خلالها مقعدًا في جمعية النواب وتولى منصب عمدة مدينة بوخارست خلال الفترة الممتدة من عام 1884 حتى عام 1886.\nباشر فليفا مسيرته السياسية مع الحزب الليبرالي الوطني الذي ساعد على تأسيسه وتمثيله أمام القضاء، ولكنه اتجه في ما بعد إلى معارضة احتكار الحزب للسلطة. حاول إنشاء حزب ثالث ودخل في مفاوضات من أجل اعتماد برامج سياسية مشتركة خاصة بقوى المعارضة المختلفة ومن بينها حزب المحافظين وجمعية جونيما في ظل الإدارات الليبرالية الوطنية المتعاقبة. ذاع صيته عندما تورط في فضيحتين كبيرتين خلال ثمانينيات القرن التاسع عشر حين أدى استهزائه بسلطة الحزب الليبرالي الوطني إلى اندلاع معارك في الشوارع ووقوع حادثتي إطلاق النار منفصلتين. اعتُبرت الجماعات الموالية لفليفا

In [ ]:
!cat "$folder_out_str"/*.ndjson | wc -l

2094596
